In [1]:
from langchain_community.utilities import SerpAPIWrapper
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Markdown
import textwrap
import os
from langchain_core.messages import HumanMessage
from langchain_core.tools import Tool
from langgraph.prebuilt import create_react_agent
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
# from langgraph.checkpoint.sqlite import SqliteSaver


def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [2]:
# using v2rayn
os.environ["HTTP_PROXY"] = "http://127.0.0.1:10809"
os.environ["HTTP_PROXYS"] = "http://127.0.0.1:10809"
# AI model
model = "gemini-1.5-flash"

In [3]:
from langchain.agents import ConversationalChatAgent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_community.callbacks import StreamlitCallbackHandler
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.runnables import RunnableConfig

In [4]:
chat = ChatGoogleGenerativeAI(model=model,
                              google_api_key=os.getenv("GOOGLE_API_KEY"))

# search = SerpAPIWrapper(serpapi_api_key=os.getenv("SERP_API_KEY"))
# search_tool = Tool(
#     name="serpapi",
#     description="A tool that uses the SerpAPI to search the web.",
#     func=search.run,
# )

# search_tool = DuckDuckGoSearchRun(name="Search")
# DuckDuckGoSearchResults
search_tool = DuckDuckGoSearchResults(name="Search")
tools = [search_tool]

# chat_with_tools = chat.bind_tools(tools)
# chat_agent = ConversationalChatAgent.from_llm_and_tools(llm=chat, tools=tools)

In [9]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(chat, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

response = agent_executor.invoke({"messages": [HumanMessage(content="What did I ask just now?")]}, config=config)

In [10]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="我叫什么")]}, config=config)
response

{'messages': [HumanMessage(content='What did I ask just now?', id='8cdb5ab1-c279-4c36-ab68-58ab554968a8'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'Search', 'arguments': '{"query": "What did I ask just now?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-a40c26a7-788a-4c15-9757-4ee4f8eb9f9a-0', tool_calls=[{'name': 'Search', 'args': {'query': 'What did I ask just now?'}, 'id': '92df20e6-b543-4c5d-8c07-5abe37c076de'}], usage_metadata={'input_tokens': 78, 'output_tokens': 19, 'total_tokens': 97}),
  Tool

In [12]:
response['messages'][-1].content

'很抱歉，我无法访问你的个人信息，包括你的名字。我是一个AI，没有记忆功能。 \n\n你想聊点别的吗？ 😊 \n'

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="我叫钱泽昊")]}, config=config)
response

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="我叫什么")]}, config=config)
response

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="帮我查一下伦敦到杜伦的距离")]}, config=config)
response

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="火车距离")]}, config=config)
response

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="我想知道直线距离")]}, config=config)
response

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="给我一些英国最近的新闻")]}, config=config)
response

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="搜索一下英国的新闻")]}, config=config)
response

In [ ]:
response['messages'][-1].content

In [ ]:
import langchain_core.messages.tool
import langchain_core.messages.ai
AI_message_list = []
tool_message_list = []


for message in response["messages"]:
    # if type(message) == 'langchain_core.messages.human.HumanMessage':
    #
    # print(type(message))
    if isinstance(message, langchain_core.messages.ai.AIMessage):
        # AI_message_list.append(message)
        if len(message.content) > 0:
            print(message.content)
            AI_message_list.append(message.content)

    if isinstance(message, langchain_core.messages.tool.ToolMessage):
        tool_message_list.append(message.content)
        print(message.content)